# 5장 재귀와 귀납법(Recursion and Induction)

__*아래 주피터 노트북은 James Brock의 [learn-you-a-haskell-notebook](https://github.com/jamesdbrock/learn-you-a-haskell-notebook)을 기본틀로 사용합니다.*__

__참고:__ 린팅(linting) 기능 끄기

* 린팅(linting): IHaskell에서 HLint라고 불리는 린터(linter)에 의해 보다 적절하다고 판단하는 표현식을 제안하는 기능
* 보다 세련된 표현식(expression)을 제안하는 도구임. 하지만 반드시 필요한 기능은 아니다
* 참조: [IHaskell의 린팅 기능 설정하기](https://github.com/gibiansky/IHaskell/wiki#opt-no-lint)

In [1]:
:opt no-lint

하스켈과 같은 함수형 프로그래밍언어에서는 재귀함수의 역할이 매우 크다. 
다른 거의 모든 프로그래밍언어에서도 재귀함수를 지원하지만 그 역할이 상대적으로 미약하다. 

__참고:__ 여러 이유가 있지만 기본적으로 C, 자바, 파이썬 등의 명령형 프로그래밍언어는 
__어떻게(*how*) 메모리를 관리할 것인가__에 보다 집중하기 때문이다. 
반면에 하스켈은 __이것의 정의는 이것이다(*is*)__라는 방식을 사용하며, 따라서
재귀함수를 대신하는 역할을 수행하는 `while` 또는 `for` 반목문(loop)을 지원하지 않는다. 

## 5.1 재귀함수(Recursive Functions)

재귀(recursion)는 함수를 정의하는 과정에서 해당 함수를 다시 활용하는 기법이며 
수학에서 자주 활용된다. 
예를 들어, 피보나찌(Fibonacci) 수열의 처음 두 값은 0과 1이며, 
연속으로 위치한 두 피보나찌 수의 합이 그 다음 피보나찌 수이다. 
이 성질을 점화식으로 표현하면 다음과 같다.

```haskell
fib(0) = 0
fib(1) = 1
fib(n) = fib(n-1) + fib(n-2)
```

하스켈에서 `fib` 함수를 아래와 같이 매우 간단하게 재귀함수로 정의할 수 있다.

__주의:__ 하스켈을 포함한 대다수의 프로그래밍언어에서는 0을 자연수로 취급하며,
리스트의 인덱스처럼 0이 첫째 항목을 가리킨다. 

In [2]:
fib :: Integer -> Integer
fib 0 = 0
fib 1 = 1
fib n = fib (n-1) + fib (n-2)

위 정의에 의하면, `n >= 2` 인 경우, `fib n`을 계산하려면 
`fib (n-1)` 과 `fib (n-2)` 를 먼저 계산해야 한다.
이와 같이 함수를 정의할 때 해당 함수를 활용하는 방식을 __재귀__(recursion)라 부른다. 

예를 들어, `fib(3)`가 계산되는 과정은 다음과 같다. 

```haskell
fib 3 = fib 2 + fib 1
      = (fib 1 + fib 0) + fib 1
      = (1 + fib 0) + fib 1
      = (1 + 0) + fib 1
      = 1 + fib 1
      = 1 + 1
      = 2
```

__참고__ 

* 소괄호를 이용하여 표현된 계산 순서에 주의하라. 
    예를 들어, 첫째 등호 오른편이 가리키는 값을 계산하기 위해 
    `fib 1`의 계산은 `fib 2`의 계산이 완료될 때까지 기다려야 한다.
* `fib` 함수는 음수에 대해서는 계산과정이 절대로 멈추지 않는다.
* `fib` 함수가 조금만 커져도 계산과정이 매우 오래 걸릴 수 있다.
    실제로 `fib n`을 계산하는 과정에서 `fib` 함수의 호출 횟수는 `n`이 커짐에 따라 기하급수적으로 늘어난다. 

### 시작단계와 재귀단계

`fib 3`의 계산과정에서 볼 수 있듯이, 0과 1이 __시작단계__(base case 또는 edge condition)의 
역할을 수행하며, 만약 이 시작단계가 없었다면 `fib 3`의 계산이 절대 끝나지 않을 것이다.
이처럼 재귀함수를 정의할 때 시작단계를 명시하는 일이 매우 중요하다. 
반면에 `n >= 2` 인 경우는 `fib` 함수를 재귀적으로 호출하는 단계, 즉, 
__재귀단계__(recursive step)라 부른다. 

정리하면, 재귀함수를 작성할 때 아래 두 사항을 고려해야 한다.

* 시작단계: 인자와 반환값을 구체적으로 지정하는 단계
* 재귀단계: 주어진 인자의 반환값을 지정하는 표현식에 해당 함수가 사용되는 단계    

예를 들어, `fib` 함수의 경우 시작단계와 재귀단계는 다음과 같다.

* 시작단계: `fib 0 = 0` 과 `fib 1 = 1`
* 재귀단계: `fib n = fib (n-1) + fib (n-2)`
    * `n`에 해당하는 피보나찌 수를 지정할 때 `fib` 함수가 두 번 사용되었음.
    * 주의: 패턴 매칭에 의해 `n >= 2`가 가정되었음

## 5.2 구조적 재귀(Structural Recursion)

재귀함수를 정의할 때 사용되는 시작단계와 재귀단계 중에서
재귀단계에 구조적 제약을 가하는 방식으로
__구조적 재귀__(structural recursion)를 정의하며,
기본 양식은 다음과 같다. 

* 시작단계: 인자와 반환값을 구체적으로 지정하는 단계
* 재귀단계: 인자를 특정 패턴으로 지정한 후 해당 패턴의 하위 패턴(subpattern)에 대한 반환값을 활용하는 단계

__하위 패턴__(subpattern)은 기존의 패턴에 하나의 구성품으로 사용된 패턴을 의미한다.
재귀함수를 구조적 재귀를 이용하여 정의하는 방법을 예제를 통해 살펴본다.

__예제:__ `maximum` 함수 구현

`maximum` 함수는 크기순으로 정렬될 수 있는 값들로 이루어진 리스트를 인자로 받아 그 중에 가장 큰 값을 반환한다. 
따라서, 항목들은 `Ord` 유형 클래스에 속하는 유형을 가져야 한다.
`maximum` 함수를 재귀로 구현하려면 시작단계와 재귀단계의 경우는 다음과 같이 정할 수 있다.

* 시작단계: 
    * 공리스트인 경우: 반환값 없음. 오류 발생해야 함.
    * 한 개의 항목으로 구성된 리스트의 경우: 해당 항목이 최대값임.
* 재귀단계: 공리스트가 아닌 경우
    * 꼬리(tail)의 최댓값과 머리(head)를 비교하여 큰 값을 반환함.
    
재귀단계에서 사용되는 '꼬리(tail)의 최댓값'이 바로 구조적 재귀에 해당하며
실제로 구현하면 다음과 같다. 

__주의:__ `maximum` 함수는 이미 정의되어 있기에 여기서는 아포스트로피를 사용하여 새로운 이름으로 정의한다.
이후 소개되는 예제에서도 동일한 방식을 사용하여 기존에 정의된 함수를 재정의한다.

In [3]:
maximum' :: (Ord a) => [a] -> a
maximum' [] = error "maximum of empty list"
maximum' [x] = x
maximum' (x:xs)
    | x > maxTail = x
    | otherwise = maxTail
    where maxTail = maximum' xs

첫째, 둘째 패턴 매칭은 시작단계에 해당한다.
마지막 셋째 패턴 매칭은 재귀단계이며, 두 개 이상의 항목을 갖는 원소를 나타내기 위해
`x:xs`를 패턴으로 사용하였다. 
위 정의가 구조적 재귀인 이유는 재귀단계의 반환값에 사용된 `maximum'` 함수의 인자가
`xs`인데, 이 패턴이 애초에 주어진 `x:xs`의 하위 패턴이기 때문이다.
실제로 `x:xs`는 `x`와 `xs`라는 두 개의 하위 패턴으로 구성되어 있다. 

위 함수에서는 `maximum' xs`가 가드(guards)와 더불어 `where` 절에서 `maxTail` 값을 지정하기 위해
사용되어서 구조적 재귀의 모습이 덜 선명하다.
반면에 `max` 함수를 이용하는 아래 정의는 구조적 재귀의 활용을 보다 명확하게 보여준다. 

In [4]:
maximum' :: (Ord a) => [a] -> a
maximum' [] = error "maximum of empty list"
maximum' [x] = x
maximum' (x:xs) = max x (maximum' xs)

예를 들어, `maximum' [2,5,1]`이 재귀적으로 계산되는 과정은 다음과 같다.

```haskell
maximum' [2,5,1] = max 2 (maximum' [5,1])
                = max 2 (max 5 (maximum' [1]))
                = max 2 (max 5 1)
                = max 2 5
                = 5
```

__참고:__ 대부분의 명령형 프로그래밍언어는 패턴매칭을 지원하지 않는다.
따라서 재귀함수를 정의할 때 패턴매칭 대신에 많은 `if ... else ...` 명령문을 사용해야 하거나,
아니면 `while` 또는 `for` 반복문을 이용한다. 
예를 들어, `maximum'` 함수의 경우 `while` 또는 `for` 반복문을 이용하여 
리스트에 포함된 전체 항목을 대상으로 최댓값을 업데이트 하는 방식으로 반환값을 지정할 수 있다. 

__예제:__ `revese` 함수 구현

`reverse` 함수는 리스트에 포함된 항목들의 순서를 뒤집어서 새로운 리스트를 생성하여 반환한다.
시작단계와 재귀단계는 다음과 같이 지정할 수 있다.

* 시작단계: 공리스트인 경우이며, 뒤집을 게 전혀 없음.
* 재귀단계: 머리와 꼬리로 구분하여, 꼬리를 뒤집고, 머리를 오른편 맨 끝에 추가.

하스켈로 구현하면 다음과 같으며, 재귀단계가 구조적 재귀를 사용하였음을 바로 확인할 수 있다. 

* `x:xs`가 인자 패턴일 때, `reverse' xs` 가 반환값에 사용되었음.
* `xs`는 `x:xs`의 하위패턴임.

In [5]:
reverse' :: [a] -> [a]
reverse' [] = []
reverse' (x:xs) = reverse' xs ++ [x]

In [6]:
reverse' [1,2,3,4,5]

[5,4,3,2,1]

__예제:__ `zip` 함수 구현

`zip` 함수는 두 개의 리스트를 인자로 받아 두 리스트에서 동일한 인덱스를 갖는 값들의 쌍으로
이루어진 리스트를 생성하여 반환한다. 
단, 생성된 리스트의 길이는 짧은 리스트의 길이와 동일하다. 

예를 들어 `zip [1,2,3] ['a','b']`의 결과는 다음과 같다.

In [7]:
zip [1,2,3] ['a','b']

[(1,'a'),(2,'b')]

즉, 보다 긴 리스트의 항목 일부는 무시된다. 

시작단계는 역시 공리스트를 대상으로 한다.
다만, 인자가 두 개이므로 각각의 인자에 대해 따로따로 다뤄야 한다.
반면에, 재귀 단계는 두 인자 모두 한 개 이상의 항목을 가질 때이다. 
`zip` 함수를 직접 구현하면 다음과 같다. 

In [8]:
zip' :: [a] -> [b] -> [(a,b)]
zip' _ [] = []
zip' [] _ = []
zip' (x:xs) (y:ys) = (x,y):zip' xs ys

처음 두 패턴매칭이 시작단계이며, 두 인자 중 하나가 공리스트인 경우를 다룬다.
셋째 패턴매칭은 재귀단계이며, 구조적 재귀를 사용한다. 

* `zip' xs ys` 에 사용된 두 인자 `xs`와 `ys` 각각 `x:xs`와 `y:ys`의 하위패턴임.

예를 들어, `zip' [1,2,3] ['a','b']`이 계산되는 과정은 다음과 같다.

```haskell
zip' [1,2,3] ['a','b'] = (1,'a') : zip' [2,3] ['b']
                       = (1,'a') : (2,'b') : zip' [3] []
                       = (1,'a') : (2,'b') : []
                       = [(1,'a'),(2,'b')]                       
```

실제 실행결과와 동일하다.

In [9]:
zip' [1,2,3] ['a','b']

[(1,'a'),(2,'b')]

__예제:__ `elem` 함수 구현

`elem` 함수는 어떤 값이 어떤 리스트의 항목으로 사용되었는지 여부를 판단해주는 함수이다. 

* 시작단계: 공리스트인 경우
* 재귀단계: 공리스트가 아닌 경우

__주의:__ 리스트의 항목으로 사용되었는지 여부를 판단하려면 항목들의 동치성(equality) 여부를 
판단할 수 있어야 한다. 따라서 `Eq` 유형 클래스에 속하는 유형을 대상으로 `elem` 함수가 작동한다.

In [10]:
elem' :: (Eq a) => a -> [a] -> Bool
elem' a [] = False
elem' a (x:xs)
    | a == x    = True
    | otherwise = a `elem'` xs

재귀단계에 사용된 `elem` 함수의 인자는 `xs`이며, `x:xs`의 하위패턴이기에 구조적 재귀가 명확하다.

In [11]:
elem' 3 [1,2,3,4]

True

In [12]:
elem' 5 [1,2,3,4]

False

__예제:__ `fib` 함수

앞서 살펴본 `fib` 함수의 정의는 구조적 재귀를 따르지 않는다.
이유는 `n-1`과 `n-2` 둘 모두 패턴 변수 `n`의 하위패턴이 아니기 때문이다.
사실 `n-1`과 `n-2`가 의미상 `n` 보다 작은 값을 표현한다는 것과 하위패턴이라는 것은 
서로 별개이다.

* 다만 자연수에 제한할 경우 구조적 재귀로 볼 수 있음?
* Data.Nat 활용?

* 패턴 매칭과 구조적 재귀의 관계? 등등

In [20]:
fib :: Integer -> Integer
fib 0 = 0
fib 1 = 1
fib (succ (succ n)) = fib (succ n) + fib (n)

: 

In [22]:
fib :: Integer -> Integer
fib 0 = 0
fib 1 = 1
fib n = fib (pred n) + fib (pred (pred n))

In [30]:
import qualified Data.Type.Nat as Foo

: 

In [28]:
:t nat0

: 

In [25]:
:i Nat

: 

In [17]:
pred 3.2

2.2

In [18]:
:t succ

succ :: forall a. Enum a => a -> a

In [19]:
succ 2

3

구조적 재귀의 장점은 사용된 인자의 패턴, 즉 모양에만 의존하여 반환값을 지정한다는 것이다.
즉, 사용된 인자의 의미를 생각할 필요없이 겉모습만 보면 해당 인자의 값을 계산할 수 있다.
반면에 구조적 재귀가 아닌 경우는 사용되는 인자의 겉모습 뿐만 아니라 의미 또한 반환값을 
지정할 때 함께 고려해야 한다. 

예를 들어, ...

모든 재귀함수를 항상 구조적 재귀함수로 정의할 수는 없다.
구조적 재귀를 사용하지 않는 재귀함수는 경우에 따라 계산 실행이 언제 멈출지 알 수 없는 경우가
발생하며, 일반적으로 판단이 불가능하다 
([튜링의 정지문제 동영상](https://www.youtube.com/watch?v=92WHN-pAFCs&t=389s) 참조). 

<img src="img/halting-problem.png"/>

잠시 뒤에 콜라츠 추측(Collatz conjecture)을 설명하면서 계산실행이 언젠가는 멈추기는 하지만 
얼마나 오래 걸리는지 예측하는 방법은 아직 알려지지 않은 재귀함수를 살펴볼 것이다.

__예제__

Now that we know how to generally think recursively, let's implement a
few functions using recursion. 

First off, we'll implement `replicate`.
`replicate` takes an `Int` and some element and returns a list that has
several repetitions of the same element. 
For instance, `replicate 3 5`
returns `[5,5,5]`. Let's think about the edge condition. My guess is that
the edge condition is 0 or less. If we try to replicate something zero
times, it should return an empty list. Also for negative numbers,
because it doesn't really make sense.

In [13]:
replicate' :: (Num i, Ord i) => i -> a -> [a]
replicate' n x
    | n <= 0    = []
    | otherwise = x:replicate' (n-1) x

We used guards here instead of patterns because we're testing for a
boolean condition. 
If `n` is less than or equal to 0, return an empty
list. Otherwise return a list that has `x` as the first element and then `x`
replicated n-1 times as the tail. Eventually, the `(n-1)` part will cause
our function to reach the edge condition.

__참고:__ `Num` is not a subclass of `Ord`. That means that what constitutes
for a number doesn't really have to adhere to an ordering. So that's why
we have to specify both the `Num` and `Ord` class constraints when doing
addition or subtraction and also comparison.

__예제__

Next up, we'll implement `take`. It takes a certain number of elements
from a list. For instance, `take 3 [5,4,3,2,1]` will return `[5,4,3]`. If we
try to take 0 or less elements from a list, we get an empty list. Also
if we try to take anything from an empty list, we get an empty list.
Notice that those are two edge conditions right there. So let's write
that out:

In [14]:
take' :: (Num i, Ord i) => i -> [a] -> [a]
take' n _
    | n <= 0   = []
take' _ []     = []
take' n (x:xs) = x : take' (n-1) xs

The first pattern specifies that if we try to take a 0 or negative
number of elements, we get an empty list. Notice that we're using `_` to
match the list because we don't really care what it is in this case.

Also notice that we use a guard, but without an `otherwise` part. That
means that if `n` turns out to be more than 0, the matching will fall
through to the next pattern. The second pattern indicates that if we try
to take anything from an empty list, we get an empty list. The third
pattern breaks the list into a head and a tail. And then we state that
taking `n` elements from a list equals a list that has `x` as the head and
then a list that takes `n-1` elements from the tail as a tail. 

예를 들어, `take' 3 [4,3,2,1]`이 계산되는 과정은 다음과 같다.

```haskell
take' 3 [4,3,2,1] = 4 : take' 2 [3,2,1]
                  = 4 : 3 : take' 1 [2,1]
                  = 4 : 3 : 2 : take' 0 [1]
                  = 4 : 3 : 2 : []
                  = [4,3,2]
```

__예제__

구조적 패턴이 아닌 재귀함수 예제

* 최대공약수 구하기 알고리즘, 유클리드 호제법
* 퀵정렬(아래 소개됨)

Because Haskell supports infinite lists, our recursion doesn't really
have to have an edge condition. 
But if it doesn't have it, it will
either keep churning at something infinitely or produce an infinite data
structure, like an infinite list. 
The good thing about infinite lists
though is that we can cut them where we want. 

__예제__

예를 들어, `repeat` takes an element and returns an infinite list that just has that element. 
A recursive implementation of that is really easy, watch.

In [15]:
repeat' :: a -> [a]
repeat' x = x:repeat' x

Calling `repeat 3` will give us a list that starts with `3` and then has an
infinite amount of 3's as a tail. So calling `repeat 3` would evaluate
like `3:repeat 3`, which is `3:(3:repeat 3)`, which is `3:(3:(3:repeat 3))`,
etc. `repeat 3` will never finish evaluating, whereas `take 5 (repeat 3)`
will give us a list of five 3's. So essentially it's like doing
`replicate 5 3`.

__예제: 퀵정렬(Quick Sort)__

We have a list of items that can be sorted. Their type is an instance of
the `Ord` typeclass. And now, we want to sort them! There's a very cool
algorithm for sorting called quicksort. It's a very clever way of sorting
items. While it takes upwards of 10 lines to implement quicksort in
imperative languages, the implementation is much shorter and elegant in
Haskell.
Therefore, let's implement it here

So, the type signature is going to be
`quicksort :: (Ord a) => [a] -> [a]`.

The edge condition? Empty list, as is expected.
A sorted empty list is an empty list. 

퀵정렬 작동법:

a sorted list is a list that has all the values smaller than (or equal
to) the head of the list in front (and those values are sorted), then
comes the head of the list in the middle and then come all the values
that are bigger than the head (they're also sorted).

Notice that we
said *sorted* two times in this definition, so we'll probably have to
make the recursive call twice! 

Also notice that we defined it using the
verb *is* to define the algorithm instead of saying *do this, do that,
then do that ...*. That's the beauty of functional programming! 

How are
we going to filter the list so that we get only the elements smaller
than the head of our list and only elements that are bigger? List
comprehensions. 

In [16]:
quicksort :: (Ord a) => [a] -> [a]
quicksort [] = []
quicksort (x:xs) =
    let smallerSorted = quicksort [a | a <- xs, a <= x]
        biggerSorted = quicksort [a | a <- xs, a > x]
    in  smallerSorted ++ [x] ++ biggerSorted

Let's give it a small test run to see if it appears to behave correctly.

In [17]:
quicksort [10,2,5,3,1,6,7,4,2,3,4,8,9]

[1,2,2,3,3,4,4,5,6,7,8,9,10]

In [18]:
quicksort "the quick brown fox jumps over the lazy dog"

"        abcdeeefghhijklmnoooopqrrsttuuvwxyz"

예를 들어, `quicksort [5,1,9,4,6,7,3]`가 계산되는 과정은 아래 그림과 같다. 

<img src="img/quicksort.png" title="quicksort" style="" />

An element that is in place and won't move anymore is represented in
<span style="color:darkorange;font-weight:bold">orange</span>. If you read them from left to right, you'll see the sorted list.
Although we chose to compare all the elements to the heads, we could
have used any element to compare against. In quicksort, an element that
you compare against is called a pivot. They're in <span style="color:green;font-weight:bold;">green</span> here. We chose
the head because it's easy to get by pattern matching. The elements that
are smaller than the pivot are <span style="color:lightgreen;font-weight:bold;">light green</span> and elements larger than the
pivot are dark green. The yellowish gradient thing represents an
application of quicksort.

__참고__ 

위 설명은 완벽하지 않다.
이유는 퀵정렬 알고리즘의 기본적인 이해를 위해서는 충분한 반면에,
`quicksort` 함수가 재귀호출되는 순서는 정확하게 명시되지 않기 때문이다. 
예를 들어, 아래 단계 이후에 어떤 재귀호출이 먼저 되는지 확실하지 않다. 

```haskell
quicksort [5,1,9,4,6,7,3] 
    = quicksort [1,4,3] ++ [5] ++ quicksort [9,6,7]
```

그런데 `quicksort`의 정의에 따르면 왼편에 위치한 `quicksort [1,4,3]`이 먼저 호출된다.
반면에 `quicksort [9,6,7]`은 `quicksort [1,4,3]`의 계산이 완료될 때까지 기다려야 한다. 
즉, `quicksort [1,4,3]`이 `[1,3,4]`로 정렬 된 이후에야 `quicksort [9,6,7]`의 계산이 시작된다.
따라서 실제 계산과정을 요약하면 다음과 같다.

```haskell
quicksort [5,1,9,4,6,7,3] 
    = quicksort [1,4,3] ++ [5] ++ quicksort [9,6,7]
    = (quicksort [] ++ [1] + [4,3]) ++ [5] ++ quicksort [9,6,7]
    = ...
    = [1,3,4] ++ [5] ++ quicksort [9,6,7]
    = [1,3,4] ++ [5] ++ (quicksort [6,7] + [9] + quicksort [])
    = ...
    = [1,3,4] ++ [5] ++ [6,7,9]
    = [1,3,4,5,6,7,9]
```

앞서 설명한 내용을 좀 더 자세히 이해하기 위해 
[YouTube 퀵정렬 설명 동영상](http://www.youtube.com/watch?v=7BDzle2n47c)을 
시청할 것을 권유한다.

<img src="img/quicksort01.png" />

위 동영상은 피벗(pivot)을 임의로 선정하는 점이 앞서의 설명과 다르지만 
퀵정렬 알고리즘의 작동방식은 완전히 동일하다. 
(6분25초 정도까지만 시청해도 됨. 이후엔 자바 언어를 이용한 알고리즘 구현을 설명함.)

## 5.3 재귀와 귀납법

앞서 살펴본 모든 재귀함수 정의는 패턴 매칭을 사용한다. 

We did quite a bit of recursion so far and as you've probably noticed,
there's a pattern here. Usually you define an edge case and then you
define a function that does something between some element and the
function applied to the rest. It doesn't matter if it's a list, a tree
or any other data structure. A sum is the first element of a list plus
the sum of the rest of the list. A product of a list is the first
element of the list times the product of the rest of the list. The
length of a list is one plus the length of the tail of the list.
Ekcetera, ekcetera ...


Of course, these also have edge cases. Usually the edge case is some
scenario where a recursive application doesn't make sense. When dealing
with lists, the edge case is most often the empty list. If you're
dealing with trees, the edge case is usually a node that doesn't have
any children.

It's similar when you're dealing with numbers recursively. Usually it
has to do with some number and the function applied to that number
modified. We did the factorial function earlier and it's the product of
a number and the factorial of that number minus one. Such a recursive
application doesn't make sense with zero, because factorials are defined
only for positive integers. 

(흠...) 
Often the edge case value turns out to be an
identity. The identity for multiplication is 1 because if you multiply
something by 1, you get that something back. Also when doing sums of
lists, we define the sum of an empty list as 0 and 0 is the identity for
addition. In quicksort, the edge case is the empty list and the identity
is also the empty list, because if you add an empty list to a list, you
just get the original list back.

So when trying to think of a recursive way to solve a problem, try to
think of when a recursive solution doesn't apply and see if you can use
that as an edge case, think about identities and think about whether
you'll break apart the parameters of the function (for instance, lists
are usually broken into a head and a tail via pattern matching) and on
which part you'll use the recursive call.

## 5.4 부록: 재귀함수의 정지문제

[콜라츠 추측(Collatz Conjecture)](https://ko.wikipedia.org/wiki/콜라츠_추측)에서
언급된 함수를 이용하여 구조적 재귀를 사용하지 않는 재귀함수의 정지문제의 판단이 
매우 어렵거나 경우에 따라 판단이 불가능할 수 있음을 설명한다.

### 콜라츠 추측

콜라츠 추측에서 사용된 재귀함수는 다음과 같다.

$$
f(n) = 
\begin{cases}
1,           & n = 1 \\
f(\frac{n}{2}), & n \text{은 짝수} \\
f(3 n + 1),     &  n \text{은 1보다 큰 홀수}
\end{cases}
$$

예를 들어, $f(3)$이 계산되는 과정은 다음과 같다.

```haskell
f(3) => 3*3+1 = 10 
     => 10/2  = 5
     => 3*5+1 = 16
     => 16/2  = 8
     => 8/2   = 4
     => 4/2   = 2
     => 2/2   = 1
     => 1
```

콜라츠 추측은 함수 $f$가 임의의 양의 정수 $n$에 대해 항상 1을 반환하며 
실행을 멈춘다는 내용이다. 
하지만 이에 대한 어떠한 증명도 지금까지 알려지지 않았다.

### 콜라츠 재귀함수의 정지문제

아래 함수는 위 함수 $f$가 각 인자에 대해 재귀적으로 몇 번 호출되는가를 세어주는 함수이며
가드를 활용하여 정의되었다.

In [19]:
collatz :: Integer -> Integer
collatz n
    | n <= 1           = 1
    | (n `mod` 2) == 0 = collatz (n `div` 2) + 1
    | (n `mod` 2) == 1 = collatz (3 * n + 1) + 1

콜라츠 추측에서 주장하듯이 몇 개의 입력값을 사용해 보면 모두 실행이 끝남을 
확인할 수 있다.
사실 지금까지 콜라츠 추측이 틀렸음을 입증하는 반례가 알려지지 않았다.
하지만 임의의 수를 입력할 때 실행이 언제 멈출 것인가 또한 
앞서 언급한 대로 아직 아무도 모른다.

예를 들어, 3을 인자로 사용하면 8번 먼에 실행을 멈춘다.

In [20]:
collatz 3

8

인자 100에 대해서는 함수 $f$가 26번 호출된다.

In [21]:
collatz 100

26

인자 171에 대해서는 125번 호출된다.

In [22]:
collatz 171

125

1부터 10,000까지를 대상으로 했을 때 함수 $f$가 가장 많이 호출되는 횟수는 다음과 같다.

* 앞서 언급한 `map` 함수를 이용하여 1부터 10,000까지의 정수에 `collatz` 함수를 적용한 후
    `maximum` 함수를 이용하여 최댓값을 찾는다. 

In [23]:
collaztNumbers = map collatz [1..10000]

In [24]:
maximum collaztNumbers

262

함수 $f$를 262 번 호출하도록 하는 인자는 
`Data.List` 모듈에 포함된 `elemIndices` 함수를 이용하여 확인할 수 있다. 

* 모듈 활용법은 나중에 자세히 설명한다.
    여기서는 먼저 특정 모듈을 아래와 같이 불러온다(import)는 것만 기억해두면 된다.

In [25]:
import Data.List

이제 262가 위치한 자리의 인덱스는 6170 하나인 것으로 확인된다. 

In [26]:
elemIndices 262 collaztNumbers

[6170]

리스트 `[1..10000]` 에서 6170 인덱스를 갖는 값은 6171이다(인덱스는 0부터 시작하기 때문임).
실제로 6171을 `collatz` 함수의 인자로 사용하면 262가 반환된다.

In [27]:
collatz 6171

262